In [218]:
""" Fama French 3 Factor Model """
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import statsmodels.api as sm

In [219]:
""" Constants for data fetching """
end_date = dt.date(2021,1,30)
start_date = dt.date(end_date.year - 8, end_date.month, end_date.day)
portfolio = ['VWRL.L']

pf_returns = web.get_data_yahoo(portfolio, start_date, end_date)['Adj Close'].pct_change()

In [220]:
""" Monthly cumfreq and slice NaN at position 0 """
pf_returns_monthly = pf_returns.resample('M').agg(lambda x: (x+1).prod() - 1)

In [221]:
""" Fama French request brings dictionary so index [0] for df """
ff_factors = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)[0]

In [222]:
""" Format date index """
pf_returns_monthly.index = ff_factors.index

In [223]:
""" Now merge """
combined = pd.merge(pf_returns_monthly, ff_factors, on='Date')

In [224]:
""" Fama French uses % not fractions """
combined[['Mkt-RF', 'SMB', 'HML', 'RF']] = combined[['Mkt-RF', 'SMB', 'HML', 'RF']]/100

In [225]:
""" Excess return column """
combined['VWRL.L-RF'] = combined['VWRL.L'] - combined.RF

In [226]:
""" Now establish regression constants """
y = combined['VWRL.L-RF']
X = combined[['Mkt-RF', 'SMB', 'HML']]

X_sm = sm.add_constant(X)

In [227]:
""" Define Model 
Generally R2 is high
Mkt-RF = Beta, risk wrt market 
SMB = Cap., + means smaller stocks
HML = Growth, + means more growth 
const = Alpha, positive performance over risks, but very high P value 
"""
model = sm.OLS(y, X_sm)
results = model.fit()
results.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              VWRL.L-RF   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.554
Method:                 Least Squares   F-statistic:                     40.80
Date:                Fri, 02 Apr 2021   Prob (F-statistic):           6.45e-17
Time:                        13:38:10   Log-Likelihood:                 233.69
No. Observations:                  97   AIC:                            -459.4
Df Residuals:                      93   BIC:                            -449.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003      0.002     -0.115      0.909      -0.005       0.005
Mkt-RF         0.6357      0.059     10.815      0.000       0.519       0.752
SMB           -0.1720      0.094     -1.829      0.071      -0.359       0.015
HML           -0.0165      0.082     -0.201      0.841      -0.180       0.147
==============================================================================
Omnibus:                        4.848   Durbin-Watson:                   2.395
Prob(Omnibus):                  0.089   Jarque-Bera (JB):                4.639
Skew:                           0.334   Prob(JB):                       0.0983
Kurtosis:                       3.838   Cond. No.                         43.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""